In [15]:
%reset -f

import numpy as np
import copy
import time

from qiskit.opflow.primitive_ops import PauliSumOp
from qiskit.quantum_info import SparsePauliOp

from qiskit.primitives import Estimator as PrimitiveEstimator
from qiskit import QuantumCircuit, transpile, Aer
from qiskit.circuit import ParameterVector
import rustworkx as rx
import networkx as nx

from qiskit_nature.problems.second_quantization.lattice import Lattice
from qiskit_nature.mappers.second_quantization import LogarithmicMapper
from heisenberg_model import HeisenbergModel

# List of optimizers: https://qiskit.org/documentation/stable/0.28/apidoc/qiskit.algorithms.optimizers.html     

print("Cell Evaluated")


Cell Evaluated


# Ansatz preserving circuit with ZZ interactions

For NISQ devices, we want to keep the ansatz as simple as possible while still being able to exploit the properties of the ground state (6 spins up/down). Our choice of ansatz is able to perserve the number of excitations easy to implement on NISQ hardware (local rotations + nearest neighbour entanglement), keeps the circuit depth low, and gives good performance.

In [18]:
def cryansatz(nqubits, layers, init_bitstring): # Hardware efficient ansatz that is spin preserving.
    
    # Separate physical and "virtual" qubits (i.e. qubits used) 
    # to prevent errors about how the number of qubits don't match.
    # However, gates will only act on the number of used qubits set by the "nqubits" argument.
    
    # From here, we will use the "initial_layout" argument during transpile to map the extra qubits
    # in the circuit to the physical ones we want to avoid (i.e. 0,6,9,15)
    physical_qubits = 16
    
    assert nqubits%2 == 0
    assert nqubits <= physical_qubits
    
    print(f"{'':+^64}\n",
      f"{' Spin Preserving CRY Ansatz (Ring Topo) ':^64}",
      f"\n{'':+^64}")
    
    my_circ = QuantumCircuit(physical_qubits)
    nparams = int((nqubits/2) * layers)
    
    params = ParameterVector('θ', nparams)
    params_idx = 0
    
    for x_ in range(len(init_bitstring)):
        if init_bitstring[x_] == "1":
            my_circ.x(x_)
            
    my_circ.barrier()
    
    for l_ in range(layers):
        if l_%2 ==0:
            for q2_ in range(0, nqubits,2):
                if q2_+1 == nqubits:
                    qb1 = q2_
                    qb2 = 0
                else:
                    qb1 = q2_
                    qb2 = q2_ + 1
                    
                my_circ.cx(qb1, qb2)
                my_circ.cry(params[params_idx], qb2, qb1)
                params_idx += 1
                my_circ.cx(qb1, qb2)
                my_circ.rzz(np.pi/2, qb2, qb1)
        else:
            for q2_ in range(1, nqubits,2):
                if q2_+1 == nqubits:
                    qb1 = q2_
                    qb2 = 0
                else:
                    qb1 = q2_
                    qb2 = q2_ + 1
                    
                my_circ.cx(qb1, qb2)
                my_circ.cry(params[params_idx], qb2, qb1)
                params_idx += 1
                my_circ.cx(qb1, qb2)
                my_circ.rzz(np.pi/2, qb2, qb1)

                
        my_circ.barrier()
                
    return my_circ


def gatecount(circuit):
    return len([circuit.data[gate][0].name for gate in range(len(circuit.data))])





# Guadalupe coupling_map
coupling_map = [[12, 15], [15, 12], [12, 10], [10, 12], 
                [7, 4], [4, 7], [8, 9], [9, 8], 
                [2, 3], [3, 2], [5, 3], [3, 5], 
                [12, 13], [13, 12], [0, 1], [1, 0], 
                [11, 8], [8, 11], [7, 6], [6, 7], 
                [11, 14], [14, 11], [5, 8], [8, 5], 
                [10, 7], [7, 10], [4, 1], [1, 4], 
                [2, 1], [1, 2], [13, 14], [14, 13]]

initial_layout = [1,2,3,5,8,11,14,13,12,10,7,4,0,6,9,15]

# Last 4 qubits are not used. These unused qubits will be mapped to 0, 6, 9 15 on the device
# using initial_layout argument during the transpile stage.
drawansatz = cryansatz(12, 4, '101010101010')

drawansatz = transpile(drawansatz, 
                       initial_layout=initial_layout,
                       optimization_level=3,
                       coupling_map = coupling_map,
                       seed_transpiler = 7725,
                       basis_gates = ['cx', 'id' ,'sx','rz','x']
                     )

print(gatecount(drawansatz))
print(drawansatz.depth())

drawansatz.draw(fold=5000)



++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
              Spin Preserving CRY Ansatz (Ring Topo)              
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
539
73


global phase: 2π
                 ░                                                                                                                                                                                                                                                     ░                                                                                                                                                                                                                                                                           ░                                                                                                                                                                                                                                                         ░                                                                                                                                                                                                                                                                            ░ 
 q_12 -> 0 ──────░─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────░───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────░─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────░────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────░─
           ┌───┐ ░      ┌────┐┌────────────────┐      ┌────┐      ┌────────┐  ┌───┐      ┌────┐  ┌──────────────────┐    ┌─────────┐                   ┌────┐  ┌──────────┐   ┌────┐               ┌────────┐   ┌────┐                                                 ░      ┌───┐                                                                                          ┌─────────┐                                                                      ┌────┐   ┌──────────┐  ┌────┐                         ┌─────────┐ ┌────┐┌──────────┐ ░      ┌────┐┌─────────────────┐       ┌────┐      ┌────────┐  ┌───┐      ┌────┐  ┌───────────────────┐     ┌─────────┐                   ┌────┐  ┌──────────┐   ┌────┐               ┌────────┐   ┌────┐                                                 ░      ┌───┐                                                                                           ┌─────────┐                                                                      ┌────┐   ┌──────────┐  ┌────┐                         ┌─────────┐ ┌────┐┌──────────┐ ░ 
  q_0 -> 1 ┤ X ├─░───■──┤ √X ├┤ Rz(θ[0]/2 + π) ├──────┤ √X ├──────┤ Rz(3π) ├──┤ X ├──────┤ √X ├──┤ Rz(π - 0.5*θ[0]) ├────┤ Rz(π/2) ├──────────■────────┤ √X ├──┤ Rz(-π/2) ├───┤ √X ├────────■──────┤ Rz(-π) ├───┤ √X ├─────────────────────────────────────────────────░──────┤ X ├───────────────────────────────────────────────────────────────────────────■──────────────┤ Rz(π/2) ├────────────────────────────────────────────────────────────■─────────┤ √X ├───┤ Rz(-π/2) ├──┤ √X ├───────────────────■─────┤ Rz(π/2) ├─┤ √X ├┤ Rz(-π/2) ├─░───■──┤ √X ├┤ Rz(θ[12]/2 + π) ├───────┤ √X ├──────┤ Rz(3π) ├──┤ X ├──────┤ √X ├──┤ Rz(π - 0.5*θ[12]) ├─────┤ Rz(π/2) ├──────────■────────┤ √X ├──┤ Rz(-π/2) ├───┤ √X ├────────■──────┤ Rz(-π) ├───┤ √X ├─────────────────────────────────────────────────░──────┤ X ├───────────────────────────────

# Creating the rotosolve optimizer

https://arxiv.org/pdf/1905.09692.pdf

The cost function, $\langle H \rangle$ is periodic w.r.t. each variational parameter. While best used for circuits where the variational parameters are single qubit gates, from our attempts, the ground state can still be found albeit with more iterations as some of the variational parameters in our ansatz are still have $2\pi$ periodicity.

Additional reasons for choosing rotosolve:
 - Easy to implement: no dynamical changing of ansatz (e.g. changing gates during optimization)
 - Efficient: Uses property of cost function and analytic form to obtain minimum for each parameter at each iteration (w.r.t. other parameters, so overall ansatz might not be optimal unless multiple iterations are used), and is therefore more efficient than other gradient-free optimizers
 - Does not require hyperparameter tuning (e.g. step size) used by many gradient-based optimizers, and therefore less optimization runs overall
 - Does not require high number of shots (can obtain ground state from as low as 100 shots from testing)
 - Converges quickly: can quickly identify when cost function does not improve, and try the next starting point
 
Other optimizers we would like to try given more time: Fraxis, RotoSelect

In [19]:
def rotosolve(parameters, circuit, hamiltonian, estimator):
    
    #https://arxiv.org/pdf/1905.09692.pdf
    
    for param_idx in range(circuit.num_parameters):
        cf = estimator.run([circuit], [hamiltonian], [parameters]).result().values[0]
        
        param_plus, param_minus = copy.deepcopy(parameters), copy.deepcopy(parameters)
        param_plus[param_idx] = parameters[param_idx] + np.pi/2
        cf_plus = estimator.run([circuit], [hamiltonian], [param_plus]).result().values[0]
        
        param_minus[param_idx] = parameters[param_idx] - np.pi/2
        cf_minus = estimator.run([circuit], [hamiltonian], [param_minus]).result().values[0]
        
        parameters[param_idx] = parameters[param_idx] - np.pi/2 - np.arctan2(2*cf - cf_plus - cf_minus, cf_plus - cf_minus)

    return parameters

print("Cell Evaluated")




Cell Evaluated


# Kagome Lattice Hamiltonian

In [20]:
# Kagome unit cell
num_qubits = 16
# Edge weight
t = 1.0

# Generate graph of kagome unit cell
# Start by defining all the edges
graph_16 = rx.PyGraph(multigraph=False)
graph_16.add_nodes_from(range(num_qubits))
edge_list = [
    (1, 2, t), (2, 3, t), (3, 5, t), (5, 8, t),
    (8, 11, t), (11, 14, t), (14, 13, t), (13, 12, t),
    (12, 10, t), (10, 7, t), (7, 4, t), (4, 1, t),
    (4, 2, t), (2, 5, t), (5, 11, t), (11, 13, t),
    (13, 10, t), (10, 4, t)]
# Generate graph from the list of edges
graph_16.add_edges_from(edge_list)

# Make a Lattice from graph
kagome_unit_cell_16 = Lattice(graph_16)
# Build Hamiltonian from graph edges
heis_16 = HeisenbergModel.uniform_parameters(
    lattice=kagome_unit_cell_16,
    uniform_interaction=t,
    uniform_onsite_potential=0.0,  # No singe site external field
)

# Map from SpinOp to qubits just as before.
log_mapper = LogarithmicMapper()
ham_16 = 4 * log_mapper.map(heis_16.second_q_ops().simplify())
# Print Hamiltonian to check it's what we expect:
# 18 ZZ, 18 YY, and 18 XX terms over 16 qubits instead of over 12 qubits

print("Cell Evaluated")



Cell Evaluated


# Running the VQE using simulators


No specific starting point chosen because the Rotosolve + our choice of ansatz can find the ground state quite reliably.
Automatically saves the runs (starting point, final params, etc) that go below -17.9 to disk. 

Scipy optimizers (e.g. COBYLA) can be used to fine tune final parameters to get closer to -18 if necessary. 

For the ideal case without sampling/device noise, this is not needed. By inspecting the final parameters, one can observe that they are your typical quarters of $\pi$ (e.g. $\frac{\pi}{4}$, $\frac{\pi}{2}$, $\frac{3\pi}{4}$, $\pm \pi$, etc). Considering that we are already very close to the minimum, we can simply divide the parameters by $\pi$, round them to the nearest "nice" fractions" and then multiply them by $\pi$ again to pretty much get -18. 

With this observation, it might also be possible to improve the performance of other optimizers by letting them optimize $\theta \in (-1,1]$ and multiply their values by $\pi$ before binding them to the circuit, although this was not tested. 



In [24]:

# Coding the starting points (rng seeds) this way ensures that you're unlikely
# to reuse the same seed twice, yet lets you keep the start points in case you want to save them
# Setting randomstate to None shows that we are truly random, not cherry picking ideal starting points for submission
n_stpt = 10
rt1 = np.random.RandomState(None) 
seedlist = np.arange(99999) 
rt1.shuffle(seedlist)
stpt_seeds = {x:seedlist[x] for x in range(n_stpt)}

shots = 1000
maxiter = 100
circuit_depth = 4
init_state = '101010101010'

vqe_circ = cryansatz(12, circuit_depth, init_state)
vqe_circ = transpile(vqe_circ, 
                       initial_layout=[1,2,3,5,4,7,8,11,10,12,13,14,0,6,9,15],
                       optimization_level=3,
                       # basis_gates=['cx', 'id', 'rz', 'sx', 'x'],
                       coupling_map = coupling_map,
                       seed_transpiler = 16
                     )

estimator = PrimitiveEstimator([vqe_circ], [ham_16],  options={'shots':shots})

for stpt in range(n_stpt):
    intermediate_info = []
    thetlist = []

    ls2 = np.random.RandomState(stpt_seeds[stpt])
    var_params = ls2.uniform(-np.pi,np.pi, vqe_circ.num_parameters)
    init_cf = estimator.run([vqe_circ], [ham_16], [var_params]).result().values[0]

    intermediate_info.append(init_cf)
    thetlist.append(var_params)

    print("Begin optimization run. Number of parameters: %.d. Stpt seed: %.d, Max iter: %.d, " 
          %(vqe_circ.num_parameters, stpt_seeds[stpt], maxiter),
          "Init cf: %.5f" %(init_cf)
          )

    ggty1 = time.time()
    for it_ in range(maxiter):
        var_params = rotosolve(var_params, vqe_circ, ham_16, estimator)
        intermediate_cf = estimator.run([vqe_circ], [ham_16], [var_params]).result().values[0]
        intermediate_info.append(intermediate_cf)
        thetlist.append(var_params)


        if it_%5 == 0:
            print(f"Current iter: {it_:<3d}, CF calls: {3*vqe_circ.num_parameters*(it_+1):<2d}, Time elapsed: {(time.time() - ggty1):<3.1f}s, "
                  # f"Est time left: {est_time_remaining:<3.1f}s, Est total time: {est_tot_time:<3.1f}s"
                  f"Current cf val: {intermediate_cf:<6.3f}"
                  )


        # Terminates if CF somehow increases for more than a set number of iterations
        # Terminates if CF found is close to the ground state
        # Terminates if CF found does not improve
        if len(intermediate_info) > 10 and np.allclose(intermediate_info[-4:], sorted(intermediate_info[-4:])):
            break
        if np.isclose(intermediate_info[-1], -18):
            break
        if len(intermediate_info) > 10 and np.isclose(intermediate_info[-1], intermediate_info[-2]):
            break

    best_index = np.argmin(intermediate_info)
    best_cf = intermediate_info[best_index]
    best_params = thetlist[best_index]
    cf_calls = 3*vqe_circ.num_parameters*len(intermediate_info)

    print("Minimum energy found: %.5f, CF calls: %.d, stpt seed: %.d" 
          %(best_cf, cf_calls, stpt_seeds[stpt]))
    print(f'Time taken: (s): {time.time() - ggty1:.2f}')


    if best_cf <= -17.9:

        # Removes all the other iterations once the minimum has been found.
        intermediate_info = copy.deepcopy(intermediate_info[:best_index+1])
        thetlist = copy.deepcopy(thetlist[:best_index+1])

        paramdict = {"seed":stpt_seeds[stpt], "opt params": best_params, "opt cf": best_cf, "cflist": intermediate_info, "paramlist":thetlist}
        np.save("cryansatz-16l"+str(circuit_depth)+"-"+init_state+"-Rotosolve" +str(stpt_seeds[stpt]) + ".npy", paramdict)
        print("Saved result")
        
    
    print()


++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
              Spin Preserving CRY Ansatz (Ring Topo)              
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
Begin optimization run. Number of parameters: 24. Stpt seed: 57186, Max iter: 100,  Init cf: 1.75848
Current iter: 0  , CF calls: 72, Time elapsed: 11.8s, Current cf val: -11.576
Current iter: 5  , CF calls: 432, Time elapsed: 71.1s, Current cf val: -15.608
Current iter: 10 , CF calls: 792, Time elapsed: 130.0s, Current cf val: -17.895
Current iter: 15 , CF calls: 1152, Time elapsed: 189.2s, Current cf val: -17.924
Current iter: 20 , CF calls: 1512, Time elapsed: 248.4s, Current cf val: -17.946
Current iter: 25 , CF calls: 1872, Time elapsed: 307.5s, Current cf val: -17.946
Current iter: 30 , CF calls: 2232, Time elapsed: 367.1s, Current cf val: -17.949
Minimum energy found: -17.98165, CF calls: 2448, stpt seed: 57186
Time taken: (s): 390.80
Saved result

Begin optimization run. Number of par